<a href="https://colab.research.google.com/github/RSeth-Creator/Data/blob/dataScience/NLP_S5_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Evaluation matrices 
1.   Accuracy = (Predicted Correctly)/(Total No. of Observation)
2.   Precision = (Predicted spam that are actually spam)/(total predicted as spam)

1.   Recall = (Predicted as spam that are actually spam)/(Total that are actually spam)






Ensemble Techniques:

1.   Techniques that creates multiple models and then combines them to provide better results than any of the single model individually
2.   Aggregating the opinion of many rather than the isolated opinion of one 


1.   Random Forest : Ensemble learning method that constructts a collection of decision trees and the aggregates the predictions of each tree  to determine the final prediction. by the voting of the output list of all  the decesion tree we can predict the outcome . it can handle ordinal & continuous data  & can handle missing value & outlier, less likely to overfit






In [2]:
#Mounting the drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## IMport required libraries

In [4]:
import nltk
import pandas as pd 
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import string 
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
ps=nltk.PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Import data from fil

In [5]:
data = pd.read_csv("drive/My Drive/SMSSpamCollection.tsv",sep = '\t', header = None)
data.columns = ['labels', 'body_text']
data.head()


,labels,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


## Create new feature (Number od character present in the string)

In [6]:
# length of character = len(string)- count of white-spaces
data['body_len']=data['body_text'].apply(lambda x :len(x)-x.count(" "))


## Create new feature (% of punctuation present in the string)

In [7]:

#Creating a function to fin dout the % of punctuation present in the text
def count_punct(text):
  count = sum([1 for char in text if char in string.punctuation])
  return round(count/(len(text) - text.count(" ")), 3)*100
    
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

data.head(2)

,labels,body_text,body_len,punct%
0,ham,I've been searching for the right words to tha...,160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,128,4.7


## Create a function to clean the text 

In [8]:
def clean_text(text):
  text = "".join([word.lower() for word in text if word not in string.punctuation])#Punctuation removal
  tokens = re.split('\W+', text) #tokenize
  text = [ps.stem(word) for word in tokens if word not in stopwords] # Stopwords removal
  return text

## Apply the vectorizer in the text to find out the score 

In [9]:
tfidf_vect  = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
#count_vect  = CountVectorizer(analyzer=clean_text)
#X_tfidf = count_vect.fit_transform(data['body_text'])

##  create the feature matrix kept the label seperate

In [10]:
X_features = pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis= 1)

In [11]:
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,8067,8068,8069,8070,8071,8072,8073,8074,8075,8076,8077,8078,8079,8080,8081,8082,8083,8084,8085,8086,8087,8088,8089,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Explore Random Forest using Cross- Validation

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score

In [13]:
rf = RandomForestClassifier(n_jobs=-1)# By setting the n_jobs =-1 it run faster 
k_fold = KFold(n_splits=5)
cross_val_score(rf,X_features,data['labels'],cv = k_fold, scoring='accuracy',n_jobs = -1)
#cross_val_score(rf(classifier),X_features(feature matrices),data['labels'](target),cv = k_fold, scoring='accuracy',n-jobs = -1)

array([0.97576302, 0.97755835, 0.97396768, 0.9640611 , 0.97124888])

## Random Forest with Hold-Out Testset 

In [14]:
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.model_selection import train_test_split

In [15]:
#Splitting the dataset into train and test
X_train,X_test,y_train, y_test = train_test_split(X_features,data['labels'],test_size= 0.2)

In [16]:
#customiza the existing random forest classifier for the use
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs=-1)

In [17]:
#Fit the rf model with the training data 
rf_model = rf.fit(X_train,y_train)

In [18]:
#Which features has the most importance in the random forest classifier
sorted(zip(rf_model.feature_importances_, X_train.columns),reverse =True)[0:10]

[(0.05711326526782927, 1804),
 (0.04193645939156851, 3135),
 (0.041837430441892226, 'body_len'),
 (0.03450783224274282, 7353),
 (0.024474614391816116, 4799),
 (0.021729050305971494, 2032),
 (0.01885797770017705, 6288),
 (0.01808435524543834, 5727),
 (0.013964538476929545, 2172),
 (0.013955988069124762, 7785)]

In [19]:
y_pred = rf_model.predict(X_test)
precision, recall,fscore, support = score(y_test,y_pred,pos_label='spam',average='binary')
#precision, recall,fscore, support = score(y_test,y_pred,pos_label='ham',average='binary')
#Based on the pos_label it will calculate the score 

In [20]:
print('precision: {} / Recall: {} / Accuracy: {}'.format(round(precision,3),round(recall,3),
                                                         (y_pred == y_test).sum()/len(y_pred)))

precision: 1.0 / Recall: 0.618 / Accuracy: 0.947935368043088




1.   Precision: 1 ==>All mail in the spam folder is the spam
2.   Recall ==> 61.8% of all spam that has come to your mailwas properly placed into the spam folder 

1.  Accuracy: ==>  95% of emails that comes into your mail that correctly identified as spam






## Random Forest With Grid Search 

Exhaustively search all the parameter/hyper parameter combination in a given grid to determine the best model.

In [21]:
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [22]:
#Splitting the dataset into train and test
X_train,X_test,y_train, y_test = train_test_split(X_features,data['labels'],test_size= 0.2)

In [23]:
def train_RF(n_est, depth):
  rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
  rf_model = rf.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
  print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
           n_est, depth, round(precision, 3), round(recall, 3),
           round((y_pred==y_test).sum() / len(y_pred), 3)))

In [24]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
      train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.246 / Accuracy: 0.904
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.585 / Accuracy: 0.947
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.69 / Accuracy: 0.961
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.831 / Accuracy: 0.978
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.261 / Accuracy: 0.906
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.634 / Accuracy: 0.953
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.746 / Accuracy: 0.968
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.845 / Accuracy: 0.98
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.289 / Accuracy: 0.909
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.669 / Accuracy: 0.958
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.746 / Accuracy: 0.968
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.831 / Accuracy: 0.978


**Check with other hyperparameter for the best performance of random forest⚡**